In [38]:
import oracledb, socket, os, getpass,tabulate
from sqlalchemy import create_engine,text
import pyodbc
import pandas as pd


In [39]:

def ora_conn_package():
    user =os.environ.get('oraUser')
    password=os.environ.get('orahr_pass')
    service_name = os.environ.get('orapluggabledb')
    port =  os.environ.get('oraport')
    server = socket.gethostname()
    
    conn = oracledb.connect(user=user,password=password,dsn=f'{server}/{service_name}',port=port )
    if conn is None:
        print("Could not connect to database")
    print("Connected to Oracle database successfully")                   
    return conn


In [45]:
def update_oradb_record(ora_connection):
     
     if ora_connection is not None:
          try:
               
               print("Enter your SQL statement (type 'END' on a new line to finish):")
               sql_lines = []
               while True:
                    line = input()
                    if line.strip().lower() == "end":
                         break
                    sql_lines.append(line)

# Join all lines into a single SQL command
               sql_statement = "\n".join(sql_lines).strip()
# Show the SQL back to the user
               print("\nYou entered the following SQL statement:")
               print("-" * 60)
               print(sql_statement)
               print("-" * 60)
               with ora_connection.cursor() as cursor:
                    cursor.execute(sql_statement)
                    if sql_statement.strip().lower().startswith("select"):
                         try:
                              max_rows = int(input("How many rows to disply? (0 = All)"))
                         except ValueError:
                              print("Invalid input. Defaulting to 10 rows.")
                              max_rows = 10
                         rows=cursor.fetchmany(max_rows) if max_rows > 0 else cursor.fetchall()
                         if not rows:
                              print("Query executed successfully. No records found.")
                              return
                         print(f'{"" if max_rows == 0 else f'User selected {max_rows} rows from record'}\n{tabulate.tabulate(rows,headers=[col[0] for col in cursor.description],tablefmt='heavy_grid')}')
                    else:
                         ora_connection.commit()
                         print(f"{cursor.rowcount} row(s) affected.")
          except Exception as e:
               print("An error occurred:", str(e))


In [41]:

def mssql_connection():

# preparing connection parameters
  server =      socket.gethostname()
  database =    os.environ.get('sqldb')
  Driver =      os.environ.get('sql_con_driver')
  user =        getpass.getuser()
  
#connection string
  connection_string = f'mssql+pyodbc://{user}@{server}/{database}?Trusted_Connection=Yes&driver={Driver}'
  engine = create_engine(connection_string)
  if engine is None:
    print("Could not connect to MSSQL server")
  print("Connected to MSSQL Server successfully")
  return engine
  

In [46]:

def update_sqldb_records(mssql_conn):
   if mssql_conn is not None:
          try:
                   
               print("Enter your SQL statement (type 'END' on a new line to finish):")
               sql_lines = []
               while True:
                  line = input()
                  if line.strip().lower() == "end":
                        break
                  sql_lines.append(line)

# Join all lines into a single SQL command
               sql_statement = "\n".join(sql_lines).strip()

# Show the SQL back to the user
               print("\nYou entered the following SQL statement:")
               print("-" * 60)
               print(sql_statement)
               print("-" * 60)
               with mssql_conn.connect() as conn:
                    result = conn.execute(sql_statement)
                    if sql_statement.strip().lower().startswith("select"):
                         try:
                              max_rows = int(input("How many rows to disply? (0 = All)"))
                         except ValueError:
                              print("Invalid input. Defaulting to 10 rows.")
                              max_rows = 10
                         rows=result.fetchmany(max_rows) if max_rows > 0 else result.fetchall()
                         if not rows:
                              print("Query executed successfully. No records found.")
                              return
                         output_df = pd.DataFrame(rows,columns=result.keys())
                         print(f'{"" if max_rows == 0 else f'User selected {max_rows} rows from record'}\n{tabulate.tabulate(output_df,headers="keys",tablefmt='heavy_grid')}')
                    else:
                         conn.commit()
                         print(f"{result.rowcount} row(s) affected.")
          except Exception as e:
               print("An error occurred:", str(e))

      

In [1]:
Prompt = input("Welcome! Which of the database do you want to access?\n ORACLE OR MSSQL SERVER")
if Prompt.strip().lower() == "oracle":
    oracle_conn = ora_conn_package()
    update_database_record =update_oradb_record(oracle_conn)    
elif Prompt.strip().lower() == "mssql server":
    mssql_conn = mssql_connection()
    update_mssqldatabase_record =update_sqldb_records(mssql_conn)
else:
    print("Invalid entry! Database do not exist")

Invalid entry! Database do not exist
